In [40]:
import pandas as pd
import random
import numpy as np
import os

In [41]:
os.getcwd()

'C:\\Users\\user\\Desktop\\뻐꾸기'

In [42]:
os.chdir("C:/Users/user/Desktop/뻐꾸기")

In [43]:
# 주간파일 불러오기
df = pd.read_csv("U_A4444.csv")
df

,등록일자,학력구분명,모집인원,공고수
0,20231007,-,○,257
1,20231014,-,○,136
2,20231021,-,○,219
3,20231028,-,○,278
4,20231104,-,○,418
...,...,...,...,...
554,20231021,학력무관,9,34
555,20231028,학력무관,9,26
556,20231104,학력무관,9,19
557,20231028,학력무관,90,1


In [44]:
# 모집인원 중 대체할 것 제외하기
ooo=["○","○○","○○○","0","00","000","10+","09","9","99","999"]
df = df[df['모집인원'].isin(ooo) == False]
df = df[df['등록일자'] != 20200104]
df = df.dropna(axis=0)
df['모집인원'] = df['모집인원'].apply(lambda x : int(x))
df

,등록일자,학력구분명,모집인원,공고수
11,20231007,-,1,1
12,20231014,-,1,1
13,20231021,-,1,1
14,20231028,-,1,2
15,20231021,-,5,2
...,...,...,...,...
549,20231028,학력무관,8,9
550,20231104,학력무관,8,25
551,20231104,학력무관,80,10
557,20231028,학력무관,90,1


In [45]:
# 자리수 생성하기
df['자리수'] = np.floor(np.log10(df['모집인원']))+1
df['자리수'] = df['자리수'].astype(int)
df

,등록일자,학력구분명,모집인원,공고수,자리수
11,20231007,-,1,1,1
12,20231014,-,1,1,1
13,20231021,-,1,1,1
14,20231028,-,1,2,1
15,20231021,-,5,2,1
...,...,...,...,...,...
549,20231028,학력무관,8,9,1
550,20231104,학력무관,8,25,1
551,20231104,학력무관,80,10,2
557,20231028,학력무관,90,1,2


In [46]:
## 날짜 별로 중앙값 구하기
df_date_dict = {}
for n in sorted(df['등록일자'].unique()):
    df_tmp = df[df['등록일자']==n]
    df_tmp_grouped = df_tmp.groupby('모집인원').sum()[['공고수']].sort_index()
    df_tmp_grouped['누적 공고수'] = df_tmp_grouped['공고수'].cumsum()
    max_num = df_tmp_grouped['누적 공고수'].max()
    mid_num = df_tmp_grouped['누적 공고수'].max()//2
    if max_num % 2 == 0:  
        n1 = df_tmp_grouped[df_tmp_grouped['누적 공고수'] >= mid_num].index[0]
        n2 = df_tmp_grouped[df_tmp_grouped['누적 공고수'] >= (mid_num +1)].index[0]
        df_date_dict[n] = (int(n1) + int(n2)) //2
    else:  
        df_date_dict[n] = df_tmp_grouped[df_tmp_grouped['누적 공고수'] > (mid_num +1)]['누적 공고수'].index[0]   
df_date_dict

C:\Users\user\AppData\Local\Temp\ipykernel_1644\909407717.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tmp_grouped = df_tmp.groupby('모집인원').sum()[['공고수']].sort_index()
C:\Users\user\AppData\Local\Temp\ipykernel_1644\909407717.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tmp_grouped = df_tmp.groupby('모집인원').sum()[['공고수']].sort_index()
C:\Users\user\AppData\Local\Temp\ipykernel_1644\909407717.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns 

{20231007: 1, 20231014: 1, 20231021: 1, 20231028: 1, 20231104: 1}

In [47]:
e_median_week = pd.DataFrame(list(df_date_dict.items()), columns = ["등록일자","중앙값"])
e_median_week

,등록일자,중앙값
0,20231007,1
1,20231014,1
2,20231021,1
3,20231028,1
4,20231104,1


In [48]:
e_median_week['자리수'] = 0
e_median_week = e_median_week[['등록일자', '자리수', '중앙값']]
e_median_week 

,등록일자,자리수,중앙값
0,20231007,0,1
1,20231014,0,1
2,20231021,0,1
3,20231028,0,1
4,20231104,0,1


In [49]:
e_median_week.to_csv('전체중앙값.csv', encoding='euc-kr', index=False)

In [50]:
## 각 날짜별로 각 자리수별 중앙값을 구하기 
dict_by_date = {}
for d in sorted(df['등록일자'].unique()):
    df_tmp_d = df[df['등록일자']==d]
    dict_by_num = {}
    for n in sorted(df_tmp_d['자리수'].unique()):
        df_tmp = df_tmp_d[df_tmp_d['자리수']==n]
        df_tmp_grouped = df_tmp.groupby('모집인원').sum()[['공고수']].sort_index()
        df_tmp_grouped['누적 공고수'] = df_tmp_grouped['공고수'].cumsum()
        max_num = df_tmp_grouped['누적 공고수'].max()
        mid_num = df_tmp_grouped['누적 공고수'].max()//2
        
        if max_num %2 == 0:
            n1 = df_tmp_grouped[df_tmp_grouped['누적 공고수'] >= mid_num].index[0]
            n2 = df_tmp_grouped[df_tmp_grouped['누적 공고수'] >= mid_num+1].index[0]
            dict_by_num[n] = (int(n1)+int(n2)) // 2
        else:
            dict_by_num[n] = df_tmp_grouped[df_tmp_grouped['누적 공고수'] >= (mid_num +1)].index[0]
    dict_by_date[d]=dict_by_num

C:\Users\user\AppData\Local\Temp\ipykernel_1644\1687847077.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tmp_grouped = df_tmp.groupby('모집인원').sum()[['공고수']].sort_index()
C:\Users\user\AppData\Local\Temp\ipykernel_1644\1687847077.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tmp_grouped = df_tmp.groupby('모집인원').sum()[['공고수']].sort_index()
C:\Users\user\AppData\Local\Temp\ipykernel_1644\1687847077.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only colum

In [51]:
result = pd.DataFrame(dict_by_date).transpose()
result

,1,2,3
20231007,1,20,100
20231014,1,20,148
20231021,1,15,100
20231028,1,15,100
20231104,1,15,100


In [52]:
result_reshaped = pd.DataFrame()
for col in result.columns:
    df_tmp = result[[col]]
    df_tmp['자리수'] = col

    df_tmp.columns = ['중앙값', '자리수']

    result_reshaped = pd.concat([result_reshaped,df_tmp],axis=0)

result_reshaped = result_reshaped.dropna().reset_index().rename({'index':'등록일자'},axis=1).groupby(['등록일자','자리수']).min()
result_reshaped

C:\Users\user\AppData\Local\Temp\ipykernel_1644\3547933261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['자리수'] = col
C:\Users\user\AppData\Local\Temp\ipykernel_1644\3547933261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['자리수'] = col
C:\Users\user\AppData\Local\Temp\ipykernel_1644\3547933261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

중앙값
등록일자     자리수     
20231007 1      1
         2     20
         3    100
20231014 1      1
         2     20
         3    148
20231021 1      1
         2     15
         3    100
20231028 1      1
         2     15
         3    100
20231104 1      1
         2     15
         3    100

In [53]:
# 저장
result_reshaped.to_csv('중앙값.csv', encoding='euc-kr')

In [54]:
e_median_week 

,등록일자,자리수,중앙값
0,20231007,0,1
1,20231014,0,1
2,20231021,0,1
3,20231028,0,1
4,20231104,0,1


In [55]:
result_reshaped

중앙값
등록일자     자리수     
20231007 1      1
         2     20
         3    100
20231014 1      1
         2     20
         3    148
20231021 1      1
         2     15
         3    100
20231028 1      1
         2     15
         3    100
20231104 1      1
         2     15
         3    100

In [56]:
result_reshaped=pd.read_csv('중앙값.csv', encoding='euc-kr')
result_reshaped

,등록일자,자리수,중앙값
0,20231007,1,1
1,20231007,2,20
2,20231007,3,100
3,20231014,1,1
4,20231014,2,20
5,20231014,3,148
6,20231021,1,1
7,20231021,2,15
8,20231021,3,100
9,20231028,1,1


In [57]:
e_medi= pd.concat([e_median_week, result_reshaped])
e_medi = e_medi.sort_values(by=["등록일자", "자리수"])
e_medi

,등록일자,자리수,중앙값
0,20231007,0,1
0,20231007,1,1
1,20231007,2,20
2,20231007,3,100
1,20231014,0,1
3,20231014,1,1
4,20231014,2,20
5,20231014,3,148
2,20231021,0,1
6,20231021,1,1


In [58]:
e_medi.to_csv('전체중앙값.csv',  encoding='euc-kr', index=False )